In [ ]:
# %% [markdown]
# # FASE 3: Análisis de Valor
# ## Persona A - Análisis de Frecuencia y Sentimiento

# %%
# Instalación de dependencias adicionales
# !pip install textblob wordcloud

# %%
import sys
import os
sys.path.append('../scripts')

# %%
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from wordcloud import WordCloud
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %%
# Cargar datos procesados
print("Cargando datos procesados...")
df = pd.read_csv('../data/processed/reviews_for_analysis.csv')
print(f"Dataset cargado: {len(df)} reseñas")
print(f"Columnas disponibles: {list(df.columns)}")

# %%
# 1. ANÁLISIS DE FRECUENCIA DE PALABRAS
print("\n" + "="*60)
print("1. ANÁLISIS DE FRECUENCIA DE PALABRAS")
print("="*60)

# Combinar todos los textos sin stopwords
all_text = ' '.join(df['texto_sin_stopwords'].dropna().astype(str))

# Tokenizar y contar frecuencia
tokens = all_text.split()
word_freq = Counter(tokens)

print(f"Total de palabras únicas: {len(word_freq)}")
print(f"Total de tokens: {len(tokens)}")

# Top 20 palabras más frecuentes
top_n = 20
top_words = word_freq.most_common(top_n)

print(f"\nTop {top_n} palabras más frecuentes:")
print("-" * 40)
for i, (word, count) in enumerate(top_words, 1):
    print(f"{i:2d}. {word:20s} : {count:4d} apariciones")

# %%
# Análisis por n-gramas (bigramas y trigramas)
print("\nANÁLISIS DE N-GRAMAS")
print("-" * 40)

from nltk import ngrams

# Crear bigramas
bigrams = list(ngrams(tokens, 2))
bigram_freq = Counter(bigrams)

# Top 10 bigramas
print("\nTop 10 bigramas más frecuentes:")
for i, (bigram, count) in enumerate(bigram_freq.most_common(10), 1):
    print(f"{i:2d}. {' '.join(bigram):30s} : {count:3d}")

# Crear trigramas
trigrams = list(ngrams(tokens, 3))
trigram_freq = Counter(trigrams)

# Top 10 trigramas
print("\nTop 10 trigramas más frecuentes:")
for i, (trigram, count) in enumerate(trigram_freq.most_common(10), 1):
    print(f"{i:2d}. {' '.join(trigram):40s} : {count:3d}")

# %%
# Análisis de frecuencia por puntuación
print("\nANÁLISIS DE FRECUENCIA POR PUNTUACIÓN")
print("-" * 40)

if 'puntuacion' in df.columns and df['puntuacion'].notna().any():
    # Separar reseñas positivas (4-5 estrellas) vs negativas (1-2 estrellas)
    df_pos = df[df['puntuacion'] >= 4]
    df_neg = df[df['puntuacion'] <= 2]
    
    print(f"Reseñas positivas (4-5 estrellas): {len(df_pos)}")
    print(f"Reseñas negativas (1-2 estrellas): {len(df_neg)}")
    
    # Combinar textos por categoría
    text_pos = ' '.join(df_pos['texto_sin_stopwords'].dropna().astype(str))
    text_neg = ' '.join(df_neg['texto_sin_stopwords'].dropna().astype(str))
    
    # Contar frecuencias
    tokens_pos = text_pos.split()
    tokens_neg = text_neg.split()
    
    freq_pos = Counter(tokens_pos)
    freq_neg = Counter(tokens_neg)
    
    # Mostrar palabras más características de cada categoría
    print("\nTop 5 palabras en reseñas POSITIVAS:")
    for word, count in freq_pos.most_common(5):
        print(f"  {word:15s}: {count:3d}")
    
    print("\nTop 5 palabras en reseñas NEGATIVAS:")
    for word, count in freq_neg.most_common(5):
        print(f"  {word:15s}: {count:3d}")

# %%
# 2. ANÁLISIS DE SENTIMIENTO
print("\n" + "="*60)
print("2. ANÁLISIS DE SENTIMIENTO")
print("="*60)

def analyze_sentiment_textblob(text):
    """Analiza sentimiento usando TextBlob"""
    if not isinstance(text, str) or not text.strip():
        return {'polarity': 0, 'subjectivity': 0, 'sentiment': 'neutral'}
    
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    
    # Clasificar sentimiento
    if polarity > 0.1:
        sentiment = 'positivo'
    elif polarity < -0.1:
        sentiment = 'negativo'
    else:
        sentiment = 'neutral'
    
    return {
        'polarity': polarity,
        'subjectivity': subjectivity,
        'sentiment': sentiment
    }

# Aplicar análisis de sentimiento a cada texto limpio
print("Analizando sentimiento de las reseñas...")
sentiment_results = []

for idx, text in enumerate(df['texto_limpio']):
    if idx % 10 == 0:
        print(f"  Procesando reseña {idx}/{len(df)}...")
    
    result = analyze_sentiment_textblob(str(text))
    sentiment_results.append(result)

# Añadir resultados al DataFrame
df_sentiment = df.copy()
for key in ['polarity', 'subjectivity', 'sentiment']:
    df_sentiment[key] = [result[key] for result in sentiment_results]

# %%
# Estadísticas de sentimiento
print("\nESTADÍSTICAS DE ANÁLISIS DE SENTIMIENTO")
print("-" * 40)

sentiment_counts = df_sentiment['sentiment'].value_counts()
print("\nDistribución de sentimientos:")
for sentiment, count in sentiment_counts.items():
    percentage = count / len(df_sentiment) * 100
    print(f"  {sentiment:10s}: {count:3d} reseñas ({percentage:.1f}%)")

print(f"\nPolaridad promedio: {df_sentiment['polarity'].mean():.3f}")
print(f"Subjetividad promedio: {df_sentiment['subjectivity'].mean():.3f}")

# %%
# Comparación sentimiento vs puntuación
if 'puntuacion' in df_sentiment.columns and df_sentiment['puntuacion'].notna().any():
    print("\nCOMPARACIÓN SENTIMIENTO VS PUNTUACIÓN")
    print("-" * 40)
    
    # Crear tabla cruzada
    cross_tab = pd.crosstab(df_sentiment['puntuacion'], df_sentiment['sentiment'])
    print("\nTabla cruzada (Puntuación × Sentimiento):")
    print(cross_tab)
    
    # Calcular correlación
    correlation = df_sentiment[['puntuacion', 'polarity']].corr().iloc[0, 1]
    print(f"\nCorrelación puntuación-polaridad: {correlation:.3f}")

# %%
# 3. ANÁLISIS ADICIONALES DE VALOR
print("\n" + "="*60)
print("3. ANÁLISIS ADICIONALES DE VALOR")
print("="*60)

# Análisis de emociones básico (basado en léxico de emociones)
print("\nANÁLISIS DE EMOCIONES BÁSICO")
print("-" * 40)

# Diccionario simple de emociones en español
emotion_lexicon = {
    'positivas': {
        'bien', 'bueno', 'excelente', 'perfecto', 'genial', 'fantástico',
        'maravilloso', 'recomiendo', 'recomendable', 'satisfecho', 'contento',
        'feliz', 'rápido', 'fácil', 'útil', 'eficiente'
    },
    'negativas': {
        'mal', 'malo', 'horrible', 'terrible', 'pésimo', 'decepcionado',
        'decepción', 'problema', 'error', 'lento', 'difícil', 'complicado',
        'caro', 'carísimo', 'devolver', 'reembolso', 'queja'
    },
    'neutras': {
        'producto', 'servicio', 'entrega', 'pedido', 'cliente', 'atención',
        'calidad', 'precio', 'tiempo', 'día', 'semana'
    }
}

def detect_emotions(text):
    """Detección básica de emociones en el texto"""
    text_lower = text.lower()
    emotions_detected = {'positivas': 0, 'negativas': 0, 'neutras': 0}
    
    for category, words in emotion_lexicon.items():
        for word in words:
            if word in text_lower:
                emotions_detected[category] += 1
    
    # Determinar emoción predominante
    max_count = max(emotions_detected.values())
    if max_count == 0:
        return 'sin_emocion_detectada'
    
    for category, count in emotions_detected.items():
        if count == max_count:
            return category

# Aplicar detección de emociones
df_sentiment['emocion_predominante'] = df_sentiment['texto_limpio'].apply(detect_emotions)

# Estadísticas de emociones
print("\nDistribución de emociones detectadas:")
emotion_counts = df_sentiment['emocion_predominante'].value_counts()
for emotion, count in emotion_counts.items():
    percentage = count / len(df_sentiment) * 100
    print(f"  {emotion:25s}: {count:3d} reseñas ({percentage:.1f}%)")

# %%
# Análisis de longitud vs sentimiento
print("\nANÁLISIS DE LONGITUD VS SENTIMIENTO")
print("-" * 40)

# Calcular longitud de textos
df_sentiment['longitud_texto'] = df_sentiment['texto_limpio'].apply(len)
df_sentiment['num_palabras'] = df_sentiment['texto_limpio'].apply(lambda x: len(str(x).split()))

# Estadísticas por sentimiento
print("\nEstadísticas por tipo de sentimiento:")
sentiment_groups = df_sentiment.groupby('sentiment')

for sentiment, group in sentiment_groups:
    print(f"\n{sentiment.upper()}:")
    print(f"  Número de reseñas: {len(group)}")
    print(f"  Longitud promedio: {group['longitud_texto'].mean():.0f} caracteres")
    print(f"  Palabras promedio: {group['num_palabras'].mean():.0f}")
    if 'puntuacion' in group.columns:
        print(f"  Puntuación promedio: {group['puntuacion'].mean():.1f}")

# %%
# Guardar resultados del análisis
print("\nGuardando resultados del análisis...")

# Guardar DataFrame con análisis de sentimiento
output_path = '../data/processed/reviews_with_sentiment.csv'
df_sentiment.to_csv(output_path, index=False, encoding='utf-8')
print(f"✓ Dataset con sentimiento guardado: {output_path}")

# Guardar estadísticas clave
stats = {
    'total_resenas': len(df_sentiment),
    'polaridad_promedio': float(df_sentiment['polarity'].mean()),
    'subjetividad_promedio': float(df_sentiment['subjectivity'].mean()),
    'distribucion_sentimientos': df_sentiment['sentiment'].value_counts().to_dict(),
    'top_palabras': dict(word_freq.most_common(20)),
    'correlacion_puntuacion_polaridad': float(correlation) if 'correlation' in locals() else None
}

import json
with open('../data/processed/analisis_stats.json', 'w', encoding='utf-8') as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)
print(f"✓ Estadísticas guardadas: ../data/processed/analisis_stats.json")

# %%
# Preparar datos para visualización
print("\nPreparando datos para visualización...")

# Crear dataset para visualización
viz_data = df_sentiment[[
    'titulo', 'texto_original', 'texto_limpio', 'puntuacion',
    'polarity', 'subjectivity', 'sentiment', 'emocion_predominante',
    'longitud_texto', 'num_palabras'
]].copy()

# Añadir información de frecuencia de palabras
word_freq_df = pd.DataFrame(word_freq.most_common(50), columns=['palabra', 'frecuencia'])
word_freq_df.to_csv('../data/processed/top_palabras.csv', index=False, encoding='utf-8')

print(f"✓ Top 50 palabras guardadas: ../data/processed/top_palabras.csv")

# %%
# Resumen del análisis
print("\n" + "="*80)
print("RESUMEN DEL ANÁLISIS")
print("="*80)
print(f"✓ Reseñas analizadas: {len(df_sentiment)}")
print(f"✓ Sentimiento predominante: {df_sentiment['sentiment'].value_counts().idxmax()}")
print(f"✓ Polaridad promedio: {df_sentiment['polarity'].mean():.3f}")
print(f"✓ Palabras únicas identificadas: {len(word_freq)}")
print(f"✓ Archivos generados:")
print(f"   - reviews_with_sentiment.csv (dataset completo con análisis)")
print(f"   - top_palabras.csv (frecuencia de palabras)")
print(f"   - analisis_stats.json (estadísticas clave)")
print("\nEl análisis está listo para la fase de visualización.")